# 手把手搭建 seq2seq（下）：Transformer

## 序列表示的第三种架构：Self-Attention


深度学习中，除了CNN和RNN可以对序列进行表示（编码）外，
*自注意力*（self-attention `Lin.Feng.Santos.ea.2017, Vaswani.Shazeer.Parmar.ea.2017`）
也可以对序列进行表示。


### 自注意力架构

给定一个由词元组成的输入序列$\mathbf{x}_1, \ldots, \mathbf{x}_n$，
其中任意$\mathbf{x}_i \in \mathbb{R}^d$（$1 \leq i \leq n$）。
该序列的自注意力输出为一个长度相同的序列
$\mathbf{y}_1, \ldots, \mathbf{y}_n$，其中：

$$\mathbf{y}_i = f(\mathbf{x}_i, (\mathbf{x}_1, \mathbf{x}_1), \ldots, (\mathbf{x}_n, \mathbf{x}_n)) \in \mathbb{R}^d$$



* $f$可以是任何注意力汇聚函数（Additive Attention、Scaled dot-product Attention、Multi-Head Attention）。
* 在自注意力中，QKV来自同一组输入。

### RNN/CNN/Self-Attention架构比较

比较下面几个架构，目标都是将由$n$个词元组成的序列映射到另一个长度相等的序列，其中的每个输入词元或输出词元都由$d$维向量表示。

<img src="assets/cnn-rnn-self-attention.svg" width="40%" height="40%" align="left"/>

假设输入/输出序列长度是$n$，

* CNN. 输入/输出通道数均为$d$，卷积核大小为$k$. 
* RNN. 输入/输出维度均为$d$.
* Self-Attention. query/key/value均为$n \times d$矩阵，假定采用dot-product attention.




| 架构            | 计算方式 | 计算复杂度             | 最大路径长度 |
| :-----         | :----:  | :----: | :----: |
| RNN            | 串行     | $\mathcal{O}(nd^2)$ | $\mathcal{O}(n)$ |
| CNN            | 并行     | $\mathcal{O}(knd^2)$ | $\mathcal{O}(n/k)$ |
| Self-Attention | 并行     | $\mathcal{O}(n^2d)$ | $\mathcal{O}(1)$ |



* 顺序操作会妨碍并行计算，而任意的序列位置组合之间的路径越短，则能更轻松地学习序列中的远距离依赖关系。

* 卷积神经网络和自注意力都拥有并行计算的优势。自注意力的最大路径长度最短，但其计算复杂度是关于序列长度的二次方，所以在很长的序列中计算会非常慢。


### 位置编码

* 自注意力因为并行计算而放弃了顺序操作。为了使用序列的顺序信息，可以通过在输入表示中添加*位置编码*（positional encoding），来注入绝对的或相对的位置信息。
* 位置编码可以通过学习得到也可以直接固定得到，差异不大。

下面介绍基于正弦函数和余弦函数的固定位置编码（:cite:`Vaswani.Shazeer.Parmar.ea.2017`）。

假设输入表示$\mathbf{X} \in \mathbb{R}^{n \times d}$
包含一个序列中$n$个词元的$d$维嵌入表示。
位置编码使用相同形状的位置嵌入矩阵
$\mathbf{P} \in \mathbb{R}^{n \times d}$输出$\mathbf{X} + \mathbf{P}$，
矩阵第$i$行、第$2j$列和$2j+1$列上的元素为：

$$\begin{aligned} p_{i, 2j} &= \sin\left(\frac{i}{10000^{2j/d}}\right),\\p_{i, 2j+1} &= \cos\left(\frac{i}{10000^{2j/d}}\right).\end{aligned}$$

在位置嵌入矩阵$\mathbf{P}$中，**行代表词元在序列中的位置，列代表位置编码的不同维度**。
基于三角函数的固定位置编码设计可以捕获绝对位置信息和相对位置信息。

#### 绝对位置信息

先看绝对位置$0, 1, \ldots, 7$的二进制编码。沿着编码维度
```
000
001
010
011
100
101
110
111
```
可以看出**沿着编码维度频率递减**，
基于三角函数的固定位置编码也有同样的性质。

<img src="assets/positional_encoding_heatmaps.png" width="50%" height="50%" align="left"/>

#### 相对位置信息

除了捕获绝对位置信息之外，上述的位置编码还允许模型学习得到输入序列中相对位置信息。

对于任何确定的位置偏移$\delta$，位置$i + \delta$处的位置编码可以线性投影位置$i$处的位置编码来表示。
令$\omega_j = 1/10000^{2j/d}$，对于任何确定的位置偏移$\delta$。

$$\begin{aligned}
&\begin{bmatrix} p_{i+\delta, 2j} \\  p_{i+\delta, 2j+1} \\ \end{bmatrix} \\
=&\begin{bmatrix} \sin\left((i+\delta) \omega_j\right) \\  \cos\left((i+\delta) \omega_j\right) \\ \end{bmatrix}\\
=&\begin{bmatrix} \cos(\delta \omega_j) \sin(i \omega_j) + \sin(\delta \omega_j) \cos(i \omega_j) \\  -\sin(\delta \omega_j) \sin(i \omega_j) + \cos(\delta \omega_j) \cos(i \omega_j) \\ \end{bmatrix}\\
&\begin{bmatrix} \cos(\delta \omega_j) & \sin(\delta \omega_j) \\  -\sin(\delta \omega_j) & \cos(\delta \omega_j) \\ \end{bmatrix}
\begin{bmatrix} p_{i, 2j} \\  p_{i, 2j+1} \\ \end{bmatrix}\\
\end{aligned}$$

## 基于Transformer的实现


自注意力同时具有并行计算和最短的最大路径长度这两个优势，使用自注意力来设计深度架构是很有吸引力的。

Transformer模型完全基于注意力机制，没有任何卷积层或循环神经网络层（`Vaswani.Shazeer.Parmar.ea.2017`），
Transformer最初应用于在文本数据上Seq2Seq学习，现在广泛应用在语言、视觉、语音和强化学习领域。


<img src="assets/transformer.svg" width="40%" height="40%" align="left"/>

上图展示了Transformer架构实现的Seq2Seq。
Transformer 作为Encoder-Decoder架构的一个实例，整体由Encoder和Decoder组成的。

* Transformer Encoder 由多个相同的层叠加而成。每层分别由*多头自注意力*（multi-head self-attention）层和*基于位置的前馈网络*（positionwise feed-forward network）两个子层（$\mathrm{sublayer}$）构成。
* Transformer Decoder 也是由多个相同的层叠加而成。每层分别由带有掩蔽的多头自注意力层、*编码器－解码器注意力*（encoder-decoder attention）层和基于位置的前馈网络构成。

* Transformer中的残差连接和层规范化是训练非常深度模型的重要工具。

### 残差连接与层规范化（Add & Norm）

受ResNet启发，子层间采用残差连接，并且使用广泛使用dropout和norm正则化技术。

不同于CV常采用batchnorm规范化技术,NLP任务由于通常是变长序列，通常采用layernorm规范化技术。

经过Add&Norm层，输入和输出形状不变。

### 基于位置的前馈网络（Positionwise FFN）

实现上就是两层MLP。

### Encoder Block

Transformer Encoder任何子层不会改变输入形状。

### Decoder Block

* 在训练阶段，其输出序列的所有位置（时间步）的词元都是已知的
* 在预测阶段，其输出序列的词元是逐个生成的。


* transformer是编码器－解码器架构的一个实践，尽管在实际情况中编码器或解码器可以单独使用。
* 在transformer中，多头自注意力用于表示输入序列和输出序列，不过解码器必须通过掩蔽机制来保留自回归属性。
* transformer模型中基于位置的前馈网络使用同一个多层感知机，作用是对所有序列位置的表示进行转换。


## references

* [d2l](https://zh-v2.d2l.ai/chapter_attention-mechanisms/attention-scoring-functions.html)
* [pytorch-seq2seq](https://github.com/bentrevett/pytorch-seq2seq)